In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

# Load dataset
df = pd.read_csv("../dataset/smart_home_dataset_v3.csv")

# Combine target (device + state)
df["label"] = df["device"] + "_" + df["state"]

# Define feature columns
numeric_cols = [
    "temperature", "humidity", "light_intensity", "noise_level",
    "pir_motion", "person_count", "co2_level", "gas_leak",
    "smoke_detected", "rain_detected"
]

text_col = "voice_command"

# Split
X_train, X_test, y_train, y_test = train_test_split(df[[text_col] + numeric_cols], df["label"], test_size=0.2, random_state=42)

# Transformers
preprocessor = ColumnTransformer([
    ("text", TfidfVectorizer(), text_col),
    ("num", StandardScaler(), numeric_cols)
])

# Model
svm_model = make_pipeline(preprocessor, SVC(kernel='rbf', probability=True))

# Train
svm_model.fit(X_train, y_train)

# Evaluate
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

                   precision    recall  f1-score   support

           ac_OFF       0.31      0.29      0.30        14
            ac_ON       0.74      0.76      0.75        38
 air_purifier_OFF       0.60      0.27      0.38        22
  air_purifier_ON       0.69      0.90      0.78        39
aquarium_pump_OFF       0.67      0.07      0.13        27
 aquarium_pump_ON       0.55      0.97      0.70        31
  cctv_camera_OFF       0.25      0.08      0.12        13
   cctv_camera_ON       0.70      0.90      0.79        31
 coffee_maker_OFF       0.98      1.00      0.99        45
      curtain_OFF       0.50      0.12      0.20        16
       curtain_ON       0.65      0.93      0.76        28
 dehumidifier_OFF       0.80      0.36      0.50        22
  dehumidifier_ON       0.69      0.94      0.79        33
    desk_lamp_OFF       0.44      0.44      0.44         9
     desk_lamp_ON       0.84      0.84      0.84        32
         door_OFF       0.71      0.19      0.30       

In [2]:
# Inference Function
def predict_action(command, sensors=None):
    """
    command: str (user's voice command)
    sensors: dict (optional sensor readings)
    """
    if sensors is None:
        sensors = {
            "temperature": 28.0,
            "humidity": 65.0,
            "light_intensity": 50.0,
            "noise_level": 30.0,
            "pir_motion": 1,
            "person_count": 1,
            "co2_level": 800.0,
            "gas_leak": 0,
            "smoke_detected": 0,
            "rain_detected": 0
        }

    # Create DataFrame for single input
    data = pd.DataFrame([{**sensors, "voice_command": command}])

    # Predict
    pred = svm_model.predict(data)[0]
    prob = max(svm_model.predict_proba(data)[0])

    print(f"User: {command}")
    print(f"→ Predicted action: {pred}  (confidence: {prob:.2f})")
    return pred

In [3]:
# Test Cases
predict_action("turn on the light")
predict_action("switch off the fan")
predict_action("it's too hot")
predict_action("open the window")
predict_action("close the door")
predict_action("good night")
predict_action("start the geyser")
predict_action("stop the hydroponic motor")
predict_action("is the fridge on")
predict_action("turn off everything")


User: turn on the light
→ Predicted action: light_ON  (confidence: 0.51)
User: switch off the fan
→ Predicted action: fan_ON  (confidence: 0.52)
User: it's too hot
→ Predicted action: air_purifier_ON  (confidence: 0.09)
User: open the window
→ Predicted action: window_ON  (confidence: 0.72)
User: close the door
→ Predicted action: door_ON  (confidence: 0.57)
User: good night
→ Predicted action: air_purifier_ON  (confidence: 0.09)
User: start the geyser
→ Predicted action: geyser_ON  (confidence: 0.55)
User: stop the hydroponic motor
→ Predicted action: cctv_camera_OFF  (confidence: 0.11)
User: is the fridge on
→ Predicted action: aquarium_pump_ON  (confidence: 0.08)
User: turn off everything
→ Predicted action: ac_OFF  (confidence: 0.14)


'ac_OFF'